# 搜索引擎

In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

tool = DuckDuckGoSearchRun()
tools = [tool]
tool.invoke("What's a 'node' in LangGraph?")

e:\work\2025_fufeng\learn-main\.venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


'May 15, 2024 · 天天在我耳边说 man, man, what can I say，问他是什么意思又不说。曼巴的意思是一种毒蛇，象征着科比在球场上的攻击性和坚韧不拔 “What can i say， Mamba out！” 但是经过互联网的解构，这句话成为了一句万用场景的话语，又因为科比这个球员因为直升机离世，还因为科比在打球的时候种种的梗，效果 ... Oct 31, 2019 · Anaconda是一个用于科学计算的Python发行版， Anaconda支持 Linux, Mac, Windows系统，提供了包管理与环境管理的功能，可以很方便地解决多版本python并存、切换以及各种第三方包安装问题。 Anaconda利用工具/命令 conda 来进行package和environment的管理，并且已经包含了Python和相关的配套工具。 在这里解释下conda ... Jan 21, 2025 · Transformer整体结构（输入两个单词的例子） 为了能够对Transformer的流程有个大致的了解，我们举一个简单的例子，还是以之前的为例，将法语"Je suis etudiant"翻译成英文。 第一步：获取输入句子的每一个单词的表示向量 ， 由单词的Embedding和单词位置的Embedding 相加得到。 Apr 27, 2019 · 日常生活中这两个词没区别，在科学领域讨论实验数值的时候是两个不同的含义。我本人不是相关专业的，以下理解仅仅是我对到目前接受的实验训练以及数学训练的个人理解。 accuracy指的是准确度，具体的衡量方式是（测量值-真值）取绝对值。测量值就是试验结果，可能是仪器测量，可能是公式 ... Feb 25, 2014 · "ching chang chong" 一个老到不行的西方歧视字汇，主要是用来嘲笑中国同胞。 让我说说其原句，还有它是怎么来的。 对于 不会说汉语的英文使用者 而言，汉语听起来就像「ching chong」一样，其中「ch」这个音来自汉语中大量的舌冠塞擦音。 ( 由於官话中有六个音： [ts]、 [tʂ]、 [tɕ]、 [tsʰ]、 [tʂʰ ...'

In [3]:
# 必要依赖（若缺）
# pip install langchain-community duckduckgo-search

from typing import Annotated
from typing_extensions import TypedDict
from pathlib import Path
import sys, os

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import BaseMessage
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# 让 notebook 也能导入项目根目录里的 config.py
try:
    base_dir = Path(__file__).resolve().parent
except NameError:
    base_dir = Path.cwd()
PROJECT_ROOT = (base_dir / "..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from config import model_configs  # 确保里面是: base_url, model, api_key, model_provider="openai"
llm = init_chat_model(**model_configs)

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

# 绑定 DuckDuckGo 工具
ddg = DuckDuckGoSearchRun()
tools = [ddg]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# 模型若产生 tool call -> 走 tools；否则结束（tools_condition 会自动路由到 END）
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

graph = graph_builder.compile()

# 单轮问答

In [4]:
result = graph.invoke({"messages": [{"role": "user", "content": "今天北京天气如何？请搜索并给出来源"}]})
msgs = result["messages"] if isinstance(result, dict) else result.messages
assistant_msgs = [m for m in msgs if getattr(m, "type", None) == "ai" or getattr(m, "role", None) == "assistant"]
final_msg = assistant_msgs[-1] if assistant_msgs else msgs[-1]
print(final_msg.content)

e:\work\2025_fufeng\learn-main\.venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
e:\work\2025_fufeng\learn-main\.venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


**今天（2025‑08‑09）北京天气简报**

| 位置 | 天气 | 温度（最低/最高） | 风向/风速 | 备注 |
|------|------|--------------------|-----------|------|
| 北京 | 晴转小雨 | 23 °C / 30 °C | 东北风2级 | 目前气温 30 °C，雨量不大，稍有阴云，但整体多云/阳光天气。

> 资料来源：**天气网**（查询结果显示“今日天气：北京市，晴转小雨，23℃ 30℃，东北风2级，当前温度30℃”）【搜索结果 1】  

**推荐做法**  
- 如需外出，建议携带雨伞或防雨服，注意及时关注降水预报。  
- 气温较高，可根据体感适当穿戴，避免中暑。  

如需更详细的小时预报或未来几天天气，请访问天气网或使用手机天气APP获取实时数据。
